<a href="https://colab.research.google.com/github/PrettySusi/Proyecto_I/blob/main/Proyecto_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##𝗣𝗥𝗢𝗬𝗘𝗖𝝩𝗢 𝝞

In [ ]:
#IMPORTANDO LIBRERIAS

import pandas as pd

import numpy as np

import random as rn

import time

import datetime

import logging

import tqdm

import warnings

import os

!pip install unidecode

from unidecode import unidecode as u

import pathlib

from pathlib import Path


#CARGA DE BASE DE DATOS MATERIAS-SEMESTRES-CRÉDITOS

dfs = pd.read_csv('https://raw.githubusercontent.com/PrettySusi/Proyecto_I/main/Materias%2C%20semestre%20y%20cr%C3%A9ditos_m.csv',encoding='ISO-8859-1', delimiter= ';')


#HORAS TRABAJO DOCENTE

cred = dfs['CRÉDITOS']

cc=  cred.tolist()

htd=[]


#HORAS TRABAJO INDEPENDIENTE

for i in range(len(cc)):
  j=int(cc[i])

  if j == 1:
    j=16
  elif j == 2:
    j=32
  elif j == 3:
    j=64
  elif j == 4:
    j=96
  elif j == 12:
    j=0
  else:
    print()
  htd.append(j)

hti=[]

for i in range(len(cc)):
  k=int(cc[i])

  if k == 1:
    k=32
  elif k == 2:
    k=64
  elif k == 3:
    k=80
  elif k == 4:
    k=120
  elif k == 12:
    k=0
  else:
    print()
  hti.append(k)

dfs['HTD'] = htd

dfs['HTI']= hti


#CÓDIGO DE LOS CURSOS

def codigoAsignatura(texto:str):
    texto =  texto.split(' ')
    codigo = []
    for t in texto:
        if len(t)>3:
            codigo.append(t)
        else:
            pass
    if len(codigo) == 1:
        return codigo[0][:3].upper()
    elif len(codigo)== 2:
        return codigo[0][:2].upper() + codigo[1][0].upper()
    else:
        return codigo[0][0].upper() + codigo[1][0].upper() + codigo[2][0].upper()


dfs['CC']= dfs['MATERIAS'].apply(codigoAsignatura)


#BASES DE DATOS NOMBRES Y APELLIDOS

dfn= pd.read_csv('https://raw.githubusercontent.com/PrettySusi/Proyecto_I/main/NombresArgentina.csv',encoding='ISO-8859-1')

dfa= pd.read_csv('https://raw.githubusercontent.com/PrettySusi/Proyecto_I/main/ApellidosArgentina.csv',encoding='ISO-8859-1')


#FILTRACIÓN DE DATOS

dfaa= dfa.sample(len(dfn)*2)

nombres = dfn['name'].tolist()

apellidos = dfaa['lastname'].tolist()

primer= apellidos[:len(dfn)-1]

segundo= apellidos[len(dfn):len(dfn)*2]

nombres_apellidos = rn.sample(list(zip(nombres, primer, segundo)), 1000)

data = pd.DataFrame(nombres_apellidos, columns=['Nombre', '1er apellido','2ndo apellido'])

data['1er apellido']= data['1er apellido'].str.upper()

data['2ndo apellido']= data['2ndo apellido'].str.upper()

nombre_c= data['Nombre completo'] = data['Nombre'].str.cat([data['1er apellido'], data['2ndo apellido']], sep=' ')

df1=pd.DataFrame({'Nombre Completo':nombre_c})


#GENERACIÓN DE SEMESTRE

def GenerearSemestre() -> int:
    semestre = [1] * 140
    semestre.extend([2] * 130)
    semestre.extend([3] * 120)
    semestre.extend([4] * 110)
    semestre.extend([5] * 100)
    semestre.extend([6] * 100)
    semestre.extend([7] * 90)
    semestre.extend([8] * 80)
    semestre.extend([9] * 70)
    semestre.extend([10] * 60)

    rn.shuffle(semestre)
    return semestre

semestres_generados = GenerearSemestre()
df2 = pd.DataFrame({'Semestre': semestres_generados})


#UNIENDO LOS ESTUDIANTES CON SU RESPECTIVO SEMESTRE


df= df1.join(df2, lsuffix='_df1', rsuffix='_df2')


#MATRÍCULA DE ESTUDIANTES

cupo_semestre = {1: 30, 2: 30, 3: 30, 4: 25, 5: 25, 6: 25, 7: 20, 8: 20, 9: 20, 10: 10}

def generar_planeacion_academica(df, dfs, cupo_semestre):
    resultados = []
    for index, row in dfs.iterrows():
        semestre = row['SEMESTRE']
        materia = row['MATERIAS']
        creditos = row['CRÉDITOS']
        htd = row['HTD']
        hti = row['HTI']
        cc= row['CC']

        cupo_aula = cupo_semestre.get(semestre, 30)

        estudiantes_semestre = df[df['Semestre'] == semestre]
        num_estudiantes = len(estudiantes_semestre)

        #AULAS

        num_grupos = (num_estudiantes // cupo_aula) + (1 if num_estudiantes % cupo_aula > 0 else 0)

        for grupo in range(num_grupos):
            estudiantes_grupo = estudiantes_semestre.iloc[grupo*cupo_aula : (grupo+1)*cupo_aula]

            #CÓDIGO ASIGNATURAS

            codigo_asignatura = u(f"{materia[:3].upper()}{semestre}{creditos}{grupo+1}")

            nombres_estudiantes = estudiantes_grupo['Nombre Completo'].tolist()
            resultados.append(( nombres_estudiantes, materia, codigo_asignatura, htd, hti, u(cc), len(estudiantes_grupo), num_grupos))

    return resultados

r = generar_planeacion_academica(df, dfs, cupo_semestre)
dat= pd.DataFrame(r)

#ORGANIZANDO DATAFRAME FINAL

columna= ['Nombre alumno','Asignatura', 'CA', 'HTD','HTI', 'CC', 'NTE','TCA']
resultado = pd.DataFrame(r, columns=columna)

rf = []
for index, row in resultado.iterrows():
    names = row['Nombre alumno']
    for n in names:

        rf.append(row.tolist()[:-1] + [row['TCA']] + [n])


c = columna[:-1] + ['TCA', 'Nombre']
p = pd.DataFrame(rf, columns=c)

orden = ['Nombre', 'Asignatura', 'CA', 'HTD', 'HTI', 'NTE','CC',  'TCA']
planilla = p[orden]


#FECHA DE CREACIÓN

planilla['FC']= datetime.datetime.now().strftime('%Y-%m-%d')


#CREACIÓN RUTA FINAL

ns= [1,2,3,4,5,6,7,8,9,10]
materias= []

uno = dfs.loc[:,['MATERIAS',	'SEMESTRE']]

for n in ns:

  semes = uno[dfs['SEMESTRE']==n]
  materias.append(semes)

l=[]
for i in range(0,10):
  f= materias[i]['MATERIAS'].tolist()
  l.append(f)

semestres=['Semestre 1','Semestre 2','Semestre 3','Semestre 4', 'Semestre 5','Semestre 6', 'Semestre 7','Semestre 8','Semestre 9', 'Semestre 10']

for sem in semestres:
  si= semestres.index(sem) +1
  mats = l[si - 1]

  for m in mats:
    a= planilla[planilla['Asignatura'] == m]

    ruta= Path('Ruta final') / sem / m
    ruta.mkdir(parents= True, exist_ok= True)

    codigo_curso = dfs.loc[dfs['MATERIAS'] == m, 'CC'].iloc[0]

    csv = ruta / f"{u(codigo_curso)}-{m.replace(' ', '').capitalize()}-{si}.csv"
    a.to_csv(csv, index=False)

    excel = ruta / f"{u(codigo_curso)}-{m.replace(' ', '').capitalize()}-{si}.xlsx"
    a.to_excel(excel, index=False)
